In [30]:
import pandas as pd
import numpy as np
import os
import time
# from parameter_tuning_CV import RunCrossValidation, TestTunedKernels

In [8]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# %matplotlib inline
from sklearn.metrics import r2_score
from scipy.optimize import curve_fit
from scipy.optimize import leastsq
import scipy.optimize as opt
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import gc

from sklearn.model_selection import LeaveOneOut
from IPython.display import display

import os
from data_preprocessing import FilteringCurves, ShowResponseCurves
from fitting_curves import FittingColumn, ShowResponseCurvesWithFitting, compute_r2_score
_FOLDER = "./data/"

### Coding Part

In [103]:
def LeaveOneOutError(kernel_model, X, y, metrics = "mse"):
    errors = []
    splitter_loo = LeaveOneOut()
#     print(splitter_loo.get_n_splits(X))
    
    for train_index, test_index in splitter_loo.split(X):
        X_train_loo, X_test_loo = X[train_index, :], X[test_index,:]
        y_train_loo, y_test_loo = y[train_index], y[test_index]
        
        model = kernel_model.fit(X_train_loo, y_train_loo)
        if metrics == "mse":
            mse = mean_squared_error(y_test_loo, model.predict(X_test_loo))
            errors.append(mse)
        elif metrics == "mae":
            mae = mean_absolute_error(y_test_loo, model.predict(X_test_loo))
            errors.append(mae)
    
    return (sum(errors)/ len(errors)) 

# gamma for RBF, laplacian, polynomial, exponential chi2 and sigmoid kernels

def RunCrossValidation(merged_df, drug_ids, number_coefficients, kernel='linear', param_tested = "alpha", 
                       param_tested_values = [], alpha=1, gamma=None, degree=3, coef0=1,
                      display_results=True):
    
    param1 = ["param_" +str(i) for i in range(10)]
    param2 = ["param" +str(i) for i in range(10)] 
    norm_response  = ["norm_cells_"+str(i) for i in range(10)]
    con_columns  = ["fd_num_"+str(i) for i in range(10)]

    not_X_columns = ["Unnamed: 0", "COSMIC_ID", "DRUG_ID"]+ param1 + param2 + norm_response + con_columns
    X_columns = set(df.columns) - set(not_X_columns)
    
    df_errors = pd.DataFrame()
    #check whether each coefficient needs its own parameters
    

    for drug_id in drug_ids:
        merged_df_i = merged_df[merged_df["DRUG_ID"]==drug_id]
        # merged_df_i has lower shape
        np.random.seed(123)
        indexes = np.random.permutation(merged_df_i.index)
        train_size = int(merged_df_i.shape[0]*0.8)
        indexes_train = indexes[:train_size]
        X_train = merged_df_i.loc[indexes_train, X_columns].values
    
        for i in range(number_coefficients):
            #check whether each coefficient needs its own parameters
            if type(alpha)==dict:
                alpha_value = alpha[i+1]
            else:
                alpha_value = alpha
                
            if type(gamma)==dict:
                gamma_value = gamma[i+1]
            else:
                gamma_value = gamma
            
            if type(degree)==dict:
                degree_value = degree[i+1]
            else:
                degree_value = degree
                
            if type(coef0)==dict:
                coef0_value = coef0[i+1]
            else:
                coef0_value = coef0
            
            y_train = merged_df_i.loc[indexes_train, "param_"+str(i+1)].values
            for param in param_tested_values:
    
                #check whether each coefficient needs its own parameters
                if param_tested == "alpha":
                    kernel_model = KernelRidge(kernel=kernel, 
                                               alpha=param, 
                                               gamma=gamma_value, 
                                               degree=degree_value, 
                                               coef0=coef0_value)
                elif param_tested == "gamma":
                    kernel_model = KernelRidge(kernel=kernel, 
                                               alpha=alpha_value, 
                                               gamma=param, 
                                               degree=degree_value,
                                               coef0=coef0_value)
                elif param_tested == "degree":
                    kernel_model = KernelRidge(kernel=kernel, 
                                               alpha=alpha_value, 
                                               gamma=gamma_value,
                                               degree=param, 
                                               coef0=coef0_value)
                elif param_tested == "coef0":
                    kernel_model = KernelRidge(kernel=kernel, 
                                               alpha=alpha_value,  
                                               gamma=gamma_value,
                                               degree=degree_value,
                                               coef0=param)
                else:
                    print("ERROR: Unknown parameters")
                
                # mse is more sensitive to different parameters choice
                mse = LeaveOneOutError(kernel_model, X_train, y_train, metrics="mse")
                df_errors.loc[drug_id, "mse_coef"+str(i+1)+"_"+str(param)] = mse
        
#     if display_results:
#         print("Tuning hyperparameter %s" % param_tested)
#         print("x0, L, k, b   y = 1/ (L + np.exp(-k*(x-x0)))+b \n")
        
    best_values = {}
    for coef in range(number_coefficients):
#         print("Results for coefficient:", coef+1)
        df_results = df_errors[["mse_coef"+str(coef+1)+"_"+str(param) for param in param_tested_values]].describe().loc[["mean", "min","max"], :]
        best_param = np.float32(df_results.loc["mean",:].idxmin().split("_")[-1])
        best_values[coef+1] = best_param
        if display_results:
            display(df_results)
            print("Coefficient %d: ,  Best %s: %.5f" % (coef+1, param_tested, best_param))
        
    del df_errors
    print("%s kernel, best values for parameter: %s" % (kernel, param_tested))
    print(best_values)
    return best_values

def TestTunedKernels(merged_df, drug_ids, number_coefficients, kernel, alpha=1, gamma=None, degree=3, coef0=1, 
                     metrics = "mse", display_results=True):
    """Training and testing Kernels with the best found hyperparameters"""
    
    param1 = ["param_" +str(i) for i in range(10)]
    param2 = ["param" +str(i) for i in range(10)] 
    norm_response  = ["norm_cells_"+str(i) for i in range(10)]
    con_columns  = ["fd_num_"+str(i) for i in range(10)]

    not_X_columns = ["Unnamed: 0", "COSMIC_ID", "DRUG_ID"]+ param1 + param2 + norm_response + con_columns
    X_columns = set(df.columns) - set(not_X_columns)
    
    df_errors_test = pd.DataFrame()

    for drug_id in drug_ids:
        # merged_df_i has lower shape
        merged_df_i = merged_df[merged_df["DRUG_ID"]==drug_id]
        
        np.random.seed(123)
        indexes = np.random.permutation(merged_df_i.index)
        train_size = int(merged_df_i.shape[0]*0.8)
        indexes_train = indexes[:train_size]
        indexes_test= indexes[train_size:]
        X_train = merged_df_i.loc[indexes_train, X_columns].values
        X_test = merged_df_i.loc[indexes_test, X_columns].values
    
        for i in range(number_coefficients):
#             param = best_param[i+1]
            y_train = merged_df_i.loc[indexes_train, "param_"+str(i+1)].values
            y_test = merged_df_i.loc[indexes_test, "param_"+str(i+1)].values
            
            #check whether each coefficient needs its own parameters
            if type(alpha)==dict:
                alpha_value = alpha[i+1]
            else:
                alpha_value = alpha
                
            if type(gamma)==dict:
                gamma_value = gamma[i+1]
            else:
                gamma_value = gamma
            
            if type(degree)==dict:
                degree_value = degree[i+1]
            else:
                degree_value = degree
                
            if type(coef0)==dict:
                coef0_value = coef0[i+1]
            else:
                coef0_value = coef0
                
            kr_lin = KernelRidge(kernel = kernel, alpha = alpha_value, gamma=gamma_value, 
                                 degree=degree_value, coef0=coef0_value)
            kr_lin.fit(X_train, y_train)
            y_pred = np.exp(kr_lin.predict(X_test))
                                
            # mse is more sensitive to different parameters choice
            if metrics == "mse":
                error = mean_squared_error(y_test, y_pred)
            elif metrics == "mae":
                error = mean_absolute_error(y_test, y_pred)
            else:
                print("ERROR: Unknown metrics")
            df_errors_test.loc[drug_id, kernel+"_mse_coef"+str(i+1)] = error
    
    df_results = df_errors_test.describe().loc[["mean", "min","max"], :]
    if display_results:
#         print("Testing %s kernel with tuned hyperparameters\n" % kernel)
#         print("Coefficients x0, L, k, b   y = 1/ (L + np.exp(-k*(x-x0)))+b") 
        display(df_results)
    return df_results

def TuneParameters(merged_df, drug_ids, number_coefficients, kernels = [], param_tested = "alpha", 
                       param_tested_values = [], alpha=1, gamma=None, degree=3, coef0=1,
                      display_results=True):
    results = {}
    for kernel in kernels:
        start_time = time.time()
        if kernel == "linear":
            best_alpha = RunCrossValidation(merged_df, drug_ids, 4, kernel=kernel, param_tested = "alpha", 
                       param_tested_values = [0.1, 0.5, 1, 5, 7, 10, 30, 50, 100, 200, 300, 500], 
                                       display_results=display_results)
            
            print("\n%s kernel: Execution time: %.3f seconds" % (kernel, (time.time() - start_time)))
            results[kernel]={}
            results[kernel]["alpha"] = best_alpha
            
        elif kernel == "polynomial":
            start_time = time.time()
            best_gamma = RunCrossValidation(merged_df, drug_ids, 4, kernel='polynomial', param_tested = "gamma", 
                                            param_tested_values = [0.00001, 0.0001, 0.01, 0.1, 1], 
                                            display_results=display_results)

            best_degree = RunCrossValidation(merged_df, drug_ids, 4, kernel='polynomial', param_tested = "degree", 
                                             gamma= best_gamma, param_tested_values = [1,2,3,4,5], 
                                             display_results=display_results)

            best_alpha = RunCrossValidation(merged_df, drug_ids, 4, kernel='polynomial', param_tested = "alpha", 
                                            gamma= best_gamma, degree = best_degree,
                                            param_tested_values = [0.001, 0.01, 0.1, 1, 5, 7], 
                                            display_results=display_results)            
            
            print("\n%s kernel: Execution time: %.3f seconds" % (kernel, (time.time() - start_time)))
            results[kernel]={}
            results[kernel]["alpha"] = best_alpha
            results[kernel]["gamma"] = best_gamma
            results[kernel]["degree"] = best_degree
            
        else: 
            start_time = time.time()
            best_gamma = RunCrossValidation(merged_df, drug_ids, 4, kernel = kernel, param_tested = "gamma", 
                                            param_tested_values = [0.00001, 0.0001, 0.01, 0.1, 0.5, 1], 
                                            display_results=display_results)

            
            best_alpha = RunCrossValidation(merged_df, drug_ids, 4, kernel=kernel, param_tested = "alpha", 
                                            param_tested_values = [0.1, 0.5, 1, 5, 7, 10, 30, 50, 100, 200, 300, 500], 
                                            display_results=display_results)
            
            best_coef0 = RunCrossValidation(merged_df, drug_ids, 4, kernel=kernel, gamma= best_gamma, 
                                            param_tested = "coef0", alpha=best_alpha,
                                            param_tested_values = [-0.1, 0, 0.1, 0.5, 1,  5, 10], 
                                            display_results=display_results)

            print("\n%s kernel: Execution time: %.3f seconds\n" % (kernel, (time.time() - start_time)))
            results[kernel]={}
            results[kernel]["alpha"] = best_alpha
            results[kernel]["gamma"] = best_gamma
            results[kernel]["coef0"] = best_coef0
            
    return  results

def TrainTestBestParameters(merged_df, drug_ids, number_coefficients, kernels =[], best_parameters_dict={}, 
                     metrics = "mse", display_results=True):
    tests={}
    for kernel in kernels:
        if kernel == "linear":
            tests["linear"] = TestTunedKernels(merged_df, drug_ids, 4, kernel='linear', 
                                               alpha=best_parameters_dict[kernel]["alpha"], 
                                               metrics = "mse", display_results=display_results)
        elif kernel == "polynomial":
            tests['polynomial'] = TestTunedKernels(merged_df, drug_ids, 4, kernel='polynomial', 
                                                   alpha=best_parameters_dict[kernel]["alpha"], 
                                                   gamma= best_parameters_dict[kernel]["gamma"], 
                                                   degree=best_parameters_dict[kernel]["degree"], 
                                                   metrics = "mse", display_results=display_results)
        else:
            tests[kernel] = TestTunedKernels(merged_df, drug_ids, 4, kernel=kernel, 
                                             alpha=best_parameters_dict[kernel]["alpha"], 
                                             gamma= best_parameters_dict[kernel]["gamma"],
                                             coef0= best_parameters_dict[kernel]["coef0"],
                                             degree=1, metrics = "mse", display_results=display_results)
    best_kernels = {}
    for i in range(4):
        test_kernels_comparison = pd.DataFrame(index=["mean", "min", "max"])
        for kernel in kernels:
            test_kernels_comparison[kernel] = tests[kernel][tests[kernel].columns[i]]
        display(test_kernels_comparison)
        best_kernels[i+1]= test_kernels_comparison.loc["mean", :].idxmin(axis=1)
        print("Coefficient: %d, best kernel: %s" % (i+1, best_kernels[i+1]))
    
    return best_kernels

### Analytical Part

In [ ]:
os.listdir("data")

**Data Preprocessing pipeline:**
    1. filter drug_profiles data 
    (123 - three stages of filtration, 23 - two stages of filtration):
        - "results/filtered_drug_profiles_123" (less data)
        - "results/filtered_drug_profiles_23" (more data)
    2. add drug features to drug data
    - "data/Drug_Features.csv" (original data)
    - "results/drug_features_with_properties2.csv" (data with pubchem properties)
    3. merged drug_profiles and drug_features
**For goog comparison:**
    filter merged data so that they have only drug with features 
    <br>for both data frames (original drug features and with added pubchem features)

In [36]:
os.listdir("results")

['drug_features_with_properties2.csv',
 'drug_features_with_properties.csv',
 'merged_drug_profiles_sigmoid4_23.csv',
 'drug_features_pubchem.csv',
 'filtered_drug_profiles_13.csv',
 'drug_features_pubchem2.csv',
 'filtered_drug_profiles_12.csv',
 'filtered_drug_profiles_23.csv',
 'filtered_drug_profiles.csv',
 'merged_drug_profiles_sigmoid4_123.csv',
 'filtered_drug_profiles_123.csv']

In [37]:
df = pd.read_csv('results/merged_drug_profiles_sigmoid4_123.csv')

In [38]:
df.head()

,Unnamed: 0,COSMIC_ID,DRUG_ID,fd_num_0,fd_num_1,fd_num_2,fd_num_3,fd_num_4,fd_num_5,fd_num_6,...,chr9:104248247-104249501(C9orf125)_HypMET,"chr9:115875199-115875738(C9orf109, C9orf110)_HypMET",chr9:123555399-123555899(FBXW2)_HypMET,chr9:140310894-140312457(EXD3)_HypMET,chr9:21974578-21975306(CDKN2A)_HypMET,chr9:35756948-35757339(MSMP)_HypMET,chr9:35791584-35791924(NPR2)_HypMET,chr9:4984543-4985630(JAK2)_HypMET,chr9:86571047-86572027(C9orf64)_HypMET,chr9:98783216-98784364(NCRNA00092)_HypMET
0,0,909704,308,0,0.111111,0.222222,0.333333,0.444444,0.555556,0.666667,...,0,0,0,0,0,0,0,0,0,0
1,1,909704,133,0,0.111111,0.222222,0.333333,0.444444,0.555556,0.666667,...,0,0,0,0,0,0,0,0,0,0
2,2,909704,328,0,0.111111,0.222222,0.333333,0.444444,0.555556,0.666667,...,0,0,0,0,0,0,0,0,0,0
3,3,909704,276,0,0.111111,0.222222,0.333333,0.444444,0.555556,0.666667,...,0,0,0,0,0,0,0,0,0,0
4,4,909704,208,0,0.111111,0.222222,0.333333,0.444444,0.555556,0.666667,...,0,0,0,0,0,0,0,0,0,0


In [39]:
conc_columns= ["fd_num_"+str(i) for i in range(10)]
response_norm = ['norm_cells_'+str(i) for i in range(10)]

gr = df.groupby(["DRUG_ID"])["COSMIC_ID"].count()
drug_ids = list(gr[gr > 70].index)
len(drug_ids)

9

In [ ]:
# kernels_to_test = ["linear", "sigmoid", "rbf", "polynomial", "additive_chi2", "laplacian"]
kernels_to_test = ["additive_chi2", "laplacian"]
results = TuneParameters(df, drug_ids[:2], 4, kernels = kernels_to_test, display_results=False)

In [105]:
print("Tuned parameters:", results)

additive_chi2 kernel, best values for parameter: gamma
{1: 1e-05, 2: 1e-05, 3: 1e-05, 4: 1e-05}
additive_chi2 kernel, best values for parameter: alpha
{1: 500.0, 2: 500.0, 3: 100.0, 4: 100.0}
additive_chi2 kernel, best values for parameter: coef0
{1: -0.1, 2: -0.1, 3: -0.1, 4: -0.1}

additive_chi2 kernel: Execution time: 238.482 seconds

laplacian kernel, best values for parameter: gamma
{1: 1e-05, 2: 1e-05, 3: 1e-04, 4: 1e-05}
laplacian kernel, best values for parameter: alpha
{1: 7.0, 2: 30.0, 3: 0.5, 4: 0.5}
laplacian kernel, best values for parameter: coef0
{1: -0.1, 2: -0.1, 3: -0.1, 4: -0.1}

laplacian kernel: Execution time: 122.763 seconds

Tuned parameters: {'additive_chi2': {'alpha': {1: 500.0, 2: 500.0, 3: 100.0, 4: 100.0}, 'gamma': {1: 1e-05, 2: 1e-05, 3: 1e-05, 4: 1e-05}, 'coef0': {1: -0.1, 2: -0.1, 3: -0.1, 4: -0.1}}, 'laplacian': {'alpha': {1: 7.0, 2: 30.0, 3: 0.5, 4: 0.5}, 'gamma': {1: 1e-05, 2: 1e-05, 3: 1e-04, 4: 1e-05}, 'coef0': {1: -0.1, 2: -0.1, 3: -0.1, 4: -0.1}}}

In [116]:
print("Tuned parameters:")
print(results)
print("\nBetter presentation:")
for key in results:
    print(key,"\t", results[key])

Tuned parameters:
{'additive_chi2': {'alpha': {1: 500.0, 2: 500.0, 3: 100.0, 4: 100.0}, 'gamma': {1: 1e-05, 2: 1e-05, 3: 1e-05, 4: 1e-05}, 'coef0': {1: -0.1, 2: -0.1, 3: -0.1, 4: -0.1}}, 'laplacian': {'alpha': {1: 7.0, 2: 30.0, 3: 0.5, 4: 0.5}, 'gamma': {1: 1e-05, 2: 1e-05, 3: 1e-04, 4: 1e-05}, 'coef0': {1: -0.1, 2: -0.1, 3: -0.1, 4: -0.1}}}

Better presentation:
additive_chi2 	 {'alpha': {1: 500.0, 2: 500.0, 3: 100.0, 4: 100.0}, 'gamma': {1: 1e-05, 2: 1e-05, 3: 1e-05, 4: 1e-05}, 'coef0': {1: -0.1, 2: -0.1, 3: -0.1, 4: -0.1}}
laplacian 	 {'alpha': {1: 7.0, 2: 30.0, 3: 0.5, 4: 0.5}, 'gamma': {1: 1e-05, 2: 1e-05, 3: 1e-04, 4: 1e-05}, 'coef0': {1: -0.1, 2: -0.1, 3: -0.1, 4: -0.1}}


In [ ]:
best_kernels = TrainTestBestParameters(merged_df, drug_ids[:4], 4, kernels = kernels_to_test, display_results=False):
print("Best Kernels:", best_kernels)

In [ ]:
neg_values = ['norm_cells_7', "norm_cells_9","norm_cells_4", "norm_cells_6","norm_cells_8","norm_cells_5"]
df[df["norm_cells_7"]<0][df.columns[:20]]